In [ ]:
#!pip install jpx_tokyo_market_prediction

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from catboost import CatBoostRegressor
#import matplotlib.pyplot as plt
#import warnings
#warnings.filterwarnings("ignore")

In [ ]:
#prices1 = pd.read_csv(r"C:\Users\Maxim\Desktop\time series\jpx-tokyo-stock-exchange-prediction\train_files\stock_prices.csv")
#prices = pd.read_csv(r"C:\Users\Maxim\Desktop\time series\jpx-tokyo-stock-exchange-prediction\supplemental_files\stock_prices.csv")

prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
prices1 = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")

In [ ]:
prices = pd.concat([prices1,prices])

In [ ]:
prices.Target = prices.Target.interpolate()

In [ ]:
#prices = #prices[['SecuritiesCode','Target','Date']]

In [ ]:
lastdays = pd.DataFrame(prices.groupby("SecuritiesCode")['Target'].mean())
def get_avg(_id_):
    return lastdays.loc[_id_]
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [ ]:
def diff(data, columnOld, columnNew):
    data[columnNew] = data[columnOld].diff()
def inv_diff(high1, high_test):
    return high1[len(high1) - len(high_test)-1:-1]+high_test

In [ ]:
#prices1 = pd.read_csv(r"C:\Users\Maxim\all_prices.csv")
#prices['Avg'] = prices1['Avg']
avg = prices['Avg']

In [ ]:
diff(prices, 'Target', 'diff_Target')
diff(prices, 'Avg', 'diff_Avg')

In [ ]:
prices.Date = pd.to_datetime(prices.Date)
prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
X=prices[["Date","SecuritiesCode","diff_Avg", "Volume"]].fillna(0)
#X=prices[["Date","SecuritiesCode","Target"]]
y=prices[["diff_Target"]].fillna(0)
codes = X.SecuritiesCode.unique()

In [ ]:
params = {
    "verbose": 1000
}
model = CatBoostRegressor(**params)
model.fit(X,y)

In [ ]:
#pd.DataFrame({'Predict':model.predict(X)}).head()

In [ ]:
model.predict(X)

In [ ]:
#avg[-1999:]

In [ ]:
#aa = avg[-1999:]+model.predict(X)[-1999:]
#aa = pd.concat([avg[:1],aa])
#aa

In [ ]:
#inv_x = inv_diff(avg, model.predict(X))
#predict = inv_x.values

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    sample_prediction["Avg"] = sample_prediction["SecuritiesCode"].apply(get_avg)
    
    avgs = sample_prediction['Avg']
    
    diff(sample_prediction, 'Avg', 'diff_Avg')
    
    
    df = sample_prediction[["Date","SecuritiesCode","diff_Avg"]].fillna(0)
    df['Volume'] = prices['Volume'].fillna(0)
    
    
    df.Date = pd.to_datetime(df.Date)
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    
    inv_X = avgs[-1999:]+model.predict(df)[-1999:]
    inv_X = pd.concat([avgs[:1],inv_X])
    predicts = inv_X.values
    print(predicts.shape)
    
    
    sample_prediction["Prediction"] = predicts
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)